In [32]:
exp_test_strs

{'DLD1_WT_BR1': 'CTGAAGCT+GCCTCTAT',
 'DLD1_WT_BR2': 'TAATGCGC +GCCTCTAT',
 'HCT116_Gemcitabine2uM_BR1': 'CGGCTATG+AGGCTATA ',
 'HCT116_Gemcitabine2uM_BR2': 'TCCGCGAA+AGGCTATA ',
 'HCT116_WT_BR1': 'CTGAAGCT+AGGCTATA ',
 'HCT116_WT_BR2': 'TAATGCGC +AGGCTATA ',
 'U2OS_WT_BR1': 'CGGCTATG+GCCTCTAT',
 'U2OS_WT_BR2': 'TCCGCGAA+GCCTCTAT'}

In [13]:
# 
from __future__ import division
import _config
from _config import OUT_PLACE, EXP_DESIGN_2955
import sys, os, fnmatch, datetime, subprocess, imp
sys.path.append('/cluster/mshen/')
import numpy as np
from collections import defaultdict
from mylib import util, compbio
import pandas as pd

import itertools as it
import re

__file__ = "b_dm_test2"

# Default params
inp_dir = OUT_PLACE + 'a_split/'
NAME = util.get_fn(__file__)
out_dir = OUT_PLACE + NAME + '/'
util.ensure_dir_exists(out_dir)

exp_design=EXP_DESIGN_2955

exp_test_strs = {}
for k,r in exp_design.iterrows():
    exp_test_strs[r.Name] = "{0}+{1}".format(r["i7 Index reads"].strip(),r["i5 Index reads"].strip())

##
# Functions
##
def match(r1,r2, h1,h2):   
  for k, v in exp_test_strs.items():
    try:
        idx = h1.index(v)
        return k,r1
    except ValueError, e:
        continue
  return "other",r1

##
# primary
##
def demultiplex(split):
  print("dm")
  #for inp_fn in [inp_dir + 'Undetermined_AH3W5GBGX9_S0_L00{0}_R1_001_{1}.fastq'.format(k2, split) for k in range(1,5)]:
  for name in list(exp_design["Name"]) + ['other']:
        util.ensure_dir_exists(out_dir + name)
        util.exists_empty_fn(out_dir + name + '/R1_%s.fa' % (split))
        util.exists_empty_fn(out_dir + name + '/R2_%s.fa' % (split))

    
  for snum, sgroup in it.groupby(
          sorted(os.listdir(inp_dir),key=lambda x:re.compile("(\d+)\.fastq").search(x).groups()[0])
      , key=lambda x:re.compile("(\d+)\.fastq").search(x).groups()[0]):
    

    if snum != split: continue
    for lnum,lgroup in it.groupby(
        sorted(list(sgroup), key=lambda x:int(re.compile("_L(\d+)").search(x).group(1)))
        , key=lambda x:int(re.compile("_L(\d+)").search(x).group(1))):
    

        
        fns = list(lgroup)
        #print fns
        #print lnum
        read_files =dict([[ int(re.compile("R(\d+)").search(e).group(1)),e] for e in fns])

        inp_fn1 = os.path.join(inp_dir, read_files[1])
        inp_fn2 = os.path.join(inp_dir, read_files[2])
        
        
        print "LANE: {0}, FILES: {1}".format(lnum, fns)
        print(inp_fn1,inp_fn2)
        break
    
        lc = util.line_count(inp_fn1)
        num_bad_q, num_tot, num_other = 0, 0, 0
        timer = util.Timer(total = lc)
        i = -1

        
        with open(inp_fn1) as f1:
          with open(inp_fn2) as f2:
            while 1:
              i+=1
              if i % 1000000 ==0 : print "{0} records, ({1}%) [{2} bad] [{3} other]".format(i/4 , 100*float(i) / lc, num_bad_q,num_other)
              try: 
                line1 = f1.next()
                line2 = f2.next()
              except StopIteration, e:
                break
            
              if i % 4 == 0:
                h1 = line1.strip()
                h2 = line2.strip()
              if i % 4 == 1:
                r1 = line1.strip()
                r2 = line2.strip()
              if i % 4 == 3:
                num_tot += 1
                qs1 = line1.strip()
                qs2 = line2.strip()
                for qs in [qs1,qs2]:
                    quals = [ord(s)-33 for s in qs]
                    if np.mean(quals) < 30:
                      num_bad_q += 1
                      continue
    
                demultiplex_id, trimmed_read = match(r1,r2, h1,h2)
    
                #raise Exception()
            
                #print demultiplex_id
                if demultiplex_id == 'other':
                  num_other += 1
              
                #break
             
                out1_fn = out_dir +  '%s/R1_%s.fa' % (demultiplex_id, split)
                with open(out1_fn, 'a') as f:
                  f.write('>' + h1[1:] + '\n' + r1 + '\n')
                  
                out2_fn = out_dir +  '%s/R2_%s.fa' % (demultiplex_id, split)
                with open(out2_fn, 'a') as f:
                  f.write('>' + h2[1:] + '\n' + r2 + '\n')
          
              #timer.update()
    
        print 'Rejected %s fraction of reads' % (num_bad_q / num_tot)
  return




In [14]:
main(split='1')


2019-06-10 12:42:52.154633
dm
LANE: 1, FILES: ['Undetermined_AH3W5GBGX9_S0_L001_R1_001_1.fastq', 'Undetermined_AH3W5GBGX9_S0_L001_R2_001_1.fastq']
('/cluster/bh0085/prj/starr_all/starr/out/a_split/Undetermined_AH3W5GBGX9_S0_L001_R1_001_1.fastq', '/cluster/bh0085/prj/starr_all/starr/out/a_split/Undetermined_AH3W5GBGX9_S0_L001_R2_001_1.fastq')
2019-06-10 12:42:52.169898
Completed in 0:00:00.015368 



'/cluster/bh0085/prj/starr_all/starr/out/b_dm_test2/'

In [4]:

##
# Main
##
@util.time_dec
def main(split = ''):
  if split == '':
    gen_qsubs()
    return

  demultiplex(split)
  return out_dir




if __name__ == '__main__':
  if len(sys.argv) > 1:
   main(sys.argv[1])
  else:
    main()


2019-06-10 12:35:57.107417
2019-06-10 12:35:57.182342
Completed in 0:00:00.075249 

